# Library

In [1]:
import optuna
import numpy as np
import pandas as pd
import json
import shap
import joblib
import os
import seaborn as sb
import seaborn as sns
import matplotlib.pyplot as plt
import dill
import re
import math
import time
import textwrap
from multiprocessing import Pool
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from functools import partial
from scipy.stats import ks_2samp, spearmanr, pointbiserialr, chi2_contingency, skewtest, normaltest, kurtosis
from datetime import datetime, timedelta
from google.cloud import bigquery
from tabulate import tabulate

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, RocCurveDisplay, PrecisionRecallDisplay, roc_auc_score, average_precision_score, recall_score, precision_score, f1_score, roc_curve, ConfusionMatrixDisplay, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

d:\anaconda\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

pd.options.display.float_format = '{:,.4f}'.format

# Additional Function

In [3]:
def transform_df (df_input: pd.DataFrame, tipe: str) -> pd.DataFrame:
    # --- Step 1: Identify columns ---
    all_cols = df_input.columns.tolist()
    base_cols = ['index', f'fitur_{tipe}']  # always present
    other_cols = ['sheet_name']

    # Detect date-like columns (numeric or datetime)
    date_cols = [c for c in all_cols if c not in base_cols + other_cols]

    # --- Step 2: Melt only the date columns ---
    df_melted = df_input.melt(
        id_vars=base_cols + other_cols,
        value_vars=date_cols,
        var_name='posisi',
        value_name='value'
    )

    # --- Step 3: Pivot so fitur_{tipe} becomes columns ---
    df_pivoted = df_melted.pivot_table(
        index=['posisi'] + other_cols,
        columns=f'fitur_{tipe}',
        values='value'
    )

    # --- Step 4: Reorder fitur_{tipe} columns using 'index' ---
    order_mapping = df_input.set_index(f'fitur_{tipe}')['index'].to_dict()
    df_pivoted = df_pivoted[sorted(df_pivoted.columns, key=lambda x: order_mapping[x])]

    # --- Step 5: Final cleanup ---
    df_pivoted = df_pivoted.reset_index()
    df_pivoted.columns.name = None

    return df_pivoted

# Import Data

## Aset

In [4]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money'
filename = "Aset.xlsx"
file_path = os.path.join(path, filename)
tipe = "aset"

# Read all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Convert to list of DataFrames with 'sheet_name' column
dataframes_list_aset = []
transformed_df_aset = []
for sheet_name, df in sheets_dict.items():
    df = df.copy()  # avoid modifying original df
    df['sheet_name'] = sheet_name
    df['fitur_aset'] = df['fitur_aset'].astype(str).str.replace(' ', '_').str.lower()
    df['fitur_aset'] = df['fitur_aset'].astype(str).str.replace('-', '_').str.lower()
    # Select all columns except the first 2
    df_new = df.iloc[:, 2:]
    # Move the last column to the first position
    cols = df_new.columns.tolist()
    reordered_cols = [cols[-1]] + cols[:-1]
    df_new = df_new[reordered_cols]
    transformed_df = transform_df(df_new, "aset")
    dataframes_list_aset.append(df_new)
    transformed_df_aset.append(transformed_df)

In [5]:
dataframes_list_aset[0]["fitur_aset"].unique()

array(['kas', 'penempatan_pada_bank_indonesia',
       'penempatan_pada_bank_lain', 'tagihan_spot_dan_derivatif/forward',
       'surat_berharga_yang_dimiliki',
       'surat_berharga_yang_dijual_dengan_janji_dibeli_kembali_(repo)',
       'tagihan_atas_surat_berharga_yang_dibeli_dengan_janji_dijual_kembali_(reverse_repo)',
       'tagihan_akseptasi', 'kredit_yang_diberikan', 'pembiayaan_syariah',
       'penyertaan_modal', 'aset_keuangan_lainnya',
       'cadangan_kerugian_penurunan_nilai_dari_aset_keuangan',
       'a.surat_berharga_yang_dimiliki',
       'b.kredit_yang_diberikan_dan_pembiayaan_syariah', 'c.lainnya',
       'aset_tidak_berwujud', 'akumulasi_amortisasi_aset_tidak_berwujud',
       'aset_tetap_dan_inventaris',
       'akumulasi_penyusutan_aset_tetap_dan_inventaris',
       'aset_non_produktif', 'a.properti_terbengkalai',
       'b.agunan_yang_diambil_alih', 'c.rekening_tunda',
       'd.aset_antar_kantor', 'aset_lainnya', 'total_aset'], dtype=object)

In [6]:
transformed_df_aset[0].columns

Index(['posisi', 'sheet_name', 'kas', 'penempatan_pada_bank_indonesia',
       'penempatan_pada_bank_lain', 'surat_berharga_yang_dimiliki',
       'tagihan_atas_surat_berharga_yang_dibeli_dengan_janji_dijual_kembali_(reverse_repo)',
       'kredit_yang_diberikan', 'aset_keuangan_lainnya',
       'cadangan_kerugian_penurunan_nilai_dari_aset_keuangan',
       'a.surat_berharga_yang_dimiliki',
       'b.kredit_yang_diberikan_dan_pembiayaan_syariah', 'c.lainnya',
       'aset_tidak_berwujud', 'akumulasi_amortisasi_aset_tidak_berwujud',
       'aset_tetap_dan_inventaris',
       'akumulasi_penyusutan_aset_tetap_dan_inventaris', 'aset_non_produktif',
       'a.properti_terbengkalai', 'b.agunan_yang_diambil_alih', 'aset_lainnya',
       'total_aset'],
      dtype='object')

In [7]:
combined_df_aset = pd.DataFrame()
for df in transformed_df_aset:
    combined_df_aset = pd.concat([combined_df_aset, df], ignore_index=True)

In [8]:
combined_df_aset

,posisi,sheet_name,kas,penempatan_pada_bank_indonesia,penempatan_pada_bank_lain,surat_berharga_yang_dimiliki,tagihan_atas_surat_berharga_yang_dibeli_dengan_janji_dijual_kembali_(reverse_repo),kredit_yang_diberikan,aset_keuangan_lainnya,cadangan_kerugian_penurunan_nilai_dari_aset_keuangan,a.surat_berharga_yang_dimiliki,b.kredit_yang_diberikan_dan_pembiayaan_syariah,c.lainnya,aset_tidak_berwujud,akumulasi_amortisasi_aset_tidak_berwujud,aset_tetap_dan_inventaris,akumulasi_penyusutan_aset_tetap_dan_inventaris,aset_non_produktif,a.properti_terbengkalai,b.agunan_yang_diambil_alih,aset_lainnya,total_aset,surat_berharga_yang_dijual_dengan_janji_dibeli_kembali_(repo),pembiayaan_syariah
0,2022-12-31,BCA Digital,23.0000,"862,490.0000","58,545.0000","5,022,545.0000","1,525,559.0000","3,239,169.0000","68,951.0000","-54,490.0000","-1,792.0000","-52,698.0000",NaN,"35,221.0000","-19,142.0000","100,755.0000","-21,184.0000","38,252.0000","15,608.0000","22,644.0000","198,157.0000","11,054,851.0000",NaN,NaN
1,2023-03-31,BCA Digital,23.0000,"742,609.0000","40,094.0000","5,014,559.0000","2,263,738.0000","3,279,170.0000","68,967.0000","-62,761.0000","-1,792.0000","-60,969.0000",NaN,"35,221.0000","-21,152.0000","101,211.0000","-25,409.0000","38,252.0000","15,608.0000","22,644.0000","169,379.0000","11,643,901.0000",NaN,NaN
2,2023-06-30,BCA Digital,23.0000,"983,573.0000","44,901.0000","4,899,481.0000","2,403,142.0000","3,923,818.0000","74,726.0000","-75,421.0000","-2,117.0000","-73,304.0000",NaN,"35,834.0000","-23,213.0000","101,305.0000","-29,682.0000","38,253.0000","15,609.0000","22,644.0000","137,724.0000","12,514,464.0000",NaN,NaN
3,2023-09-30,BCA Digital,23.0000,"912,056.0000","116,911.0000","5,247,483.0000","1,768,527.0000","4,461,324.0000","85,047.0000","-109,727.0000","-2,300.0000","-107,425.0000",-2.0000,"37,445.0000","-25,366.0000","101,668.0000","-33,993.0000","38,253.0000","15,609.0000","22,644.0000","103,052.0000","12,702,703.0000",NaN,NaN
4,2023-12-31,BCA Digital,23.0000,"942,424.0000","158,987.0000","5,295,848.0000","2,281,369.0000","4,641,960.0000","104,205.0000","-140,297.0000","-1,009.0000","-139,247.0000",-41.0000,"37,985.0000","-27,677.0000","101,684.0000","-37,597.0000","26,650.0000","6,228.0000","20,422.0000","121,164.0000","13,506,728.0000",NaN,NaN
5,2024-03-31,BCA Digital,23.0000,"1,077,858.0000","203,258.0000","6,887,430.0000","1,437,216.0000","4,516,783.0000","101,712.0000","-179,749.0000","-1,024.0000","-178,667.0000",-58.0000,"37,985.0000","-29,082.0000","102,941.0000","-40,569.0000","26,650.0000","6,228.0000","20,422.0000","195,801.0000","14,338,257.0000",NaN,NaN
6,2024-06-30,BCA Digital,23.0000,"1,177,623.0000","357,224.0000","7,410,041.0000","332,330.0000","5,340,601.0000","106,260.0000","-172,923.0000","-1,094.0000","-171,768.0000",-61.0000,"37,985.0000","-30,488.0000","103,348.0000","-43,687.0000","26,637.0000","6,228.0000","20,409.0000","151,474.0000","14,796,448.0000",NaN,NaN
7,2024-09-30,BCA Digital,23.0000,"858,834.0000","338,486.0000","8,058,309.0000","353,168.0000","5,508,393.0000","94,934.0000","-167,185.0000","-1,107.0000","-165,951.0000",-127.0000,"54,345.0000","-33,938.0000","106,998.0000","-46,975.0000","26,637.0000","6,228.0000","20,409.0000","109,238.0000","15,261,267.0000",NaN,NaN
8,2024-12-31,BCA Digital,23.0000,"884,828.0000","172,630.0000","7,829,063.0000","551,241.0000","6,520,545.0000","118,873.0000","-244,057.0000","-1,323.0000","-242,637.0000",-97.0000,"58,045.0000","-37,429.0000","111,355.0000","-50,605.0000","25,080.0000","6,105.0000","18,975.0000","114,855.0000","16,054,447.0000",NaN,NaN
9,2022-12-31,Seabank,"4,219.0000","1,827,617.0000","430,768.0000","3,807,799.0000","6,072,130.0000","15,893,317.0000","955,767.0000","-1,617,001.0000",NaN,"-1,615,867.0000","-1,134.0000","36,531.0000","-21,823.0000","638,216.0000","-247,802.0000","90,597.0000",NaN,"90,597.0000","399,425.0000","28,269,760.0000",NaN,NaN


## Rasio

In [9]:
path = r'D:\Jupyter Notebook\Pinang Interbank Call Money'
filename = "Rasio.xlsx"
file_path = os.path.join(path, filename)
tipe = "rasio"

# Read all sheets into a dictionary of DataFrames
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Convert to list of DataFrames with 'sheet_name' column
dataframes_list_rasio = []
transformed_df_rasio = []
for sheet_name, df in sheets_dict.items():
    df = df.copy()  # avoid modifying original df
    df['sheet_name'] = sheet_name
    df['fitur_rasio'] = df['fitur_rasio'].astype(str).str.replace(' ', '_').str.lower()
    df['fitur_rasio'] = df['fitur_rasio'].astype(str).str.replace('-', '_').str.lower()
    # Select all columns except the first 2
    df_new = df.iloc[:, 2:]
    # Move the last column to the first position
    cols = df_new.columns.tolist()
    reordered_cols = [cols[-1]] + cols[:-1]
    df_new = df_new[reordered_cols]
    transformed_df = transform_df(df_new, "rasio")
    dataframes_list_rasio.append(df_new)
    transformed_df_rasio.append(transformed_df)

In [10]:
transformed_df_rasio[0].columns

Index(['posisi', 'sheet_name', 'kewajiban_penyediaan_modal_minimum',
       'aset_produktif_bermasalah_dan_aset_non_produktif_bermasalah_terhadap_total_aset_produktif_dan_aset_non_produktif',
       'aset_produktif_bermasalah_terhadap__total_aset_produktif',
       'cadangan_kerugian_penurunan_nilai_(ckpn)_aset_keuangan_terhadap_aset_produktif',
       'npl_gross', 'npl_net', 'return_on_asset', 'return_on_equity',
       'net_interest_margin',
       'biaya_operasional_terhadap_pendapatan_operasional',
       'cost_to_income_ratio', 'loan_to_deposit_ratio', 'leverage_ratio',
       'i.__pihak_terkait_(pelanggaran_bmpk)',
       'ii._pihak_tidak_terkait_(pelanggaran_bmpk)',
       'i.__pihak_terkait_(pelampauan_bmpk)',
       'ii._pihak_tidak_terkait_(pelampauan_bmpk)', '__harian', '__rata_rata',
       'b.gwm_valuta_asing_(harian)',
       'posisi_devisa_neto_(pdn)_secara_keseluruhan'],
      dtype='object')

In [11]:
combined_df_rasio = pd.DataFrame()
for df in transformed_df_rasio:
    combined_df_rasio = pd.concat([combined_df_rasio, df], ignore_index=True)

In [12]:
combined_df_rasio

,posisi,sheet_name,kewajiban_penyediaan_modal_minimum,aset_produktif_bermasalah_dan_aset_non_produktif_bermasalah_terhadap_total_aset_produktif_dan_aset_non_produktif,aset_produktif_bermasalah_terhadap__total_aset_produktif,cadangan_kerugian_penurunan_nilai_(ckpn)_aset_keuangan_terhadap_aset_produktif,npl_gross,npl_net,return_on_asset,return_on_equity,net_interest_margin,biaya_operasional_terhadap_pendapatan_operasional,cost_to_income_ratio,loan_to_deposit_ratio,leverage_ratio,i.__pihak_terkait_(pelanggaran_bmpk),ii._pihak_tidak_terkait_(pelanggaran_bmpk),i.__pihak_terkait_(pelampauan_bmpk),ii._pihak_tidak_terkait_(pelampauan_bmpk),__harian,__rata_rata,b.gwm_valuta_asing_(harian),posisi_devisa_neto_(pdn)_secara_keseluruhan,a.gwm_utama_rupiah
0,2022-03-31,BCA Digital,2.2741,0.0068,0.0000,0.0018,0.0000,0.0000,-0.0138,-0.0226,0.0316,1.3312,1.4002,0.3861,0.5757,0.0000,0.0000,0.0000,0.0000,0.0769,0.0000,0.0000,0.0000,NaN
1,2022-06-30,BCA Digital,1.6872,0.0053,0.0000,0.0040,0.0000,0.0000,-0.0100,-0.0182,0.0299,1.2070,1.1035,0.4151,0.4754,0.0000,0.0000,0.0000,0.0000,0.0973,0.0000,0.0000,0.0000,NaN
2,2022-09-30,BCA Digital,1.4651,0.0042,0.0000,0.0027,0.0000,0.0000,-0.0030,-0.0061,0.0306,1.0653,1.0195,0.3617,0.4011,0.0000,0.0000,0.0000,0.0000,0.1269,0.0000,0.0000,0.0000,NaN
3,2022-12-31,BCA Digital,0.9691,0.0041,0.0003,0.0056,0.0009,0.0004,-0.0083,-0.0180,0.0336,1.1643,1.0885,0.4733,0.3570,0.0000,0.0000,0.0000,0.0000,0.1221,0.0000,0.0000,0.0000,NaN
4,2023-03-31,BCA Digital,0.9774,0.0044,0.0008,0.0059,0.0027,0.0013,0.0003,0.0008,0.0450,0.9982,0.9311,0.4389,0.3392,0.0000,0.0000,0.0000,0.0000,0.1053,0.0000,0.0000,0.0000,NaN
5,2023-06-30,BCA Digital,0.8303,0.0049,0.0016,0.0066,0.0045,0.0019,0.0008,0.0024,0.0446,0.9887,0.8867,0.4679,0.3136,0.0000,0.0000,0.0000,0.0000,0.1191,0.0000,0.0000,0.0000,NaN
6,2023-09-30,BCA Digital,0.7537,0.0053,0.0021,0.0094,0.0055,0.0019,0.0013,0.0038,0.0488,0.9828,0.8269,0.5210,0.3097,0.0000,0.0000,0.0000,0.0000,0.1191,0.0000,0.0000,0.0000,NaN
7,2023-12-31,BCA Digital,0.7148,0.0060,0.0040,0.0109,0.0110,0.0021,0.0018,0.0116,0.0536,0.9766,0.8123,0.5172,0.2874,0.0000,0.0000,0.0000,0.0000,0.1350,0.0000,0.0000,0.0000,NaN
8,2024-03-31,BCA Digital,0.6538,0.0072,0.0054,0.0119,0.0181,0.0037,0.0071,0.0227,0.0597,0.9211,0.5075,0.4501,0.2701,0.0000,0.0000,0.0000,0.0000,0.1108,0.0000,0.0000,0.0000,NaN
9,2024-06-30,BCA Digital,0.5820,0.0068,0.0051,0.0110,0.0150,0.0028,0.0074,0.0198,0.0614,0.9228,0.5590,0.5073,0.2639,0.0000,0.0000,0.0000,0.0000,0.1055,0.1039,0.0000,0.0000,NaN


# Visualisasi Data

## Aset

In [13]:
# Ensure posisi is in datetime format
combined_df_aset['posisi'] = pd.to_datetime(combined_df_aset['posisi'])

# Extract year and quarter
combined_df_aset['year'] = combined_df_aset['posisi'].dt.year
combined_df_aset['quarter'] = combined_df_aset['posisi'].dt.month.map(lambda m: f"q{(m-1)//3 + 1}")
combined_df_aset['year_quarter'] = combined_df_aset['year'].astype(str) + '_' + combined_df_aset['quarter'] 

# Create company_date: sheet_name + "_" + year + quarter
combined_df_aset['company_date'] = combined_df_aset['sheet_name'] + "_" + combined_df_aset['year'].astype(str) + "_" + combined_df_aset['quarter']

In [14]:
combined_df_aset.columns

Index(['posisi', 'sheet_name', 'kas', 'penempatan_pada_bank_indonesia',
       'penempatan_pada_bank_lain', 'surat_berharga_yang_dimiliki',
       'tagihan_atas_surat_berharga_yang_dibeli_dengan_janji_dijual_kembali_(reverse_repo)',
       'kredit_yang_diberikan', 'aset_keuangan_lainnya',
       'cadangan_kerugian_penurunan_nilai_dari_aset_keuangan',
       'a.surat_berharga_yang_dimiliki',
       'b.kredit_yang_diberikan_dan_pembiayaan_syariah', 'c.lainnya',
       'aset_tidak_berwujud', 'akumulasi_amortisasi_aset_tidak_berwujud',
       'aset_tetap_dan_inventaris',
       'akumulasi_penyusutan_aset_tetap_dan_inventaris', 'aset_non_produktif',
       'a.properti_terbengkalai', 'b.agunan_yang_diambil_alih', 'aset_lainnya',
       'total_aset',
       'surat_berharga_yang_dijual_dengan_janji_dibeli_kembali_(repo)',
       'pembiayaan_syariah', 'year', 'quarter', 'year_quarter',
       'company_date'],
      dtype='object')

In [15]:
combined_df_aset.rename(
    columns={
        "sheet_name" : "company_name"
    },
    inplace=True
)

In [16]:
# combined_df_aset.to_excel("summarized_aset.xlsx", index=False)

## Rasio

In [17]:
# Ensure posisi is in datetime format
combined_df_rasio['posisi'] = pd.to_datetime(combined_df_rasio['posisi'])

# Extract year and quarter
combined_df_rasio['year'] = combined_df_rasio['posisi'].dt.year
combined_df_rasio['quarter'] = combined_df_rasio['posisi'].dt.month.map(lambda m: f"q{(m-1)//3 + 1}")
combined_df_rasio['year_quarter'] = combined_df_rasio['year'].astype(str) + '_' + combined_df_rasio['quarter'] 
 
# Create company_date: sheet_name + "_" + year + quarter
combined_df_rasio['company_date'] = combined_df_rasio['sheet_name'] + "_" + combined_df_rasio['year'].astype(str) + combined_df_rasio['quarter']

In [18]:
combined_df_rasio.columns

Index(['posisi', 'sheet_name', 'kewajiban_penyediaan_modal_minimum',
       'aset_produktif_bermasalah_dan_aset_non_produktif_bermasalah_terhadap_total_aset_produktif_dan_aset_non_produktif',
       'aset_produktif_bermasalah_terhadap__total_aset_produktif',
       'cadangan_kerugian_penurunan_nilai_(ckpn)_aset_keuangan_terhadap_aset_produktif',
       'npl_gross', 'npl_net', 'return_on_asset', 'return_on_equity',
       'net_interest_margin',
       'biaya_operasional_terhadap_pendapatan_operasional',
       'cost_to_income_ratio', 'loan_to_deposit_ratio', 'leverage_ratio',
       'i.__pihak_terkait_(pelanggaran_bmpk)',
       'ii._pihak_tidak_terkait_(pelanggaran_bmpk)',
       'i.__pihak_terkait_(pelampauan_bmpk)',
       'ii._pihak_tidak_terkait_(pelampauan_bmpk)', '__harian', '__rata_rata',
       'b.gwm_valuta_asing_(harian)',
       'posisi_devisa_neto_(pdn)_secara_keseluruhan', 'a.gwm_utama_rupiah',
       'year', 'quarter', 'year_quarter', 'company_date'],
      dtype='object'

In [19]:
combined_df_rasio.rename(
    columns={
        "sheet_name" : "company_name"
    },
    inplace=True
)

In [ ]:
# combined_df_rasio.to_excel("summarized_rasio.xlsx", index=False)

### Single Feature Checking

In [35]:
column_to_check = 'npl_gross'

fig_box = px.box(
    combined_df_rasio,
    x=column_to_check,
    color='company_name',
    points='outliers',  # show outlier dots
    orientation='h',
    title=f'Boxplot of {column_to_check} by company_name'
)
fig_box.update_xaxes(tickformat=".02%")
fig_box.show()

In [36]:
fig_hist = px.histogram(
    combined_df_rasio, 
    x=column_to_check,
    color='company_name',
    nbins=20,
    title=f'Histogram of {column_to_check} by company_name'
)
fig_hist.update_xaxes(tickformat=".02%")
fig_hist.update_yaxes(title_text="Count of Observations")
fig_hist.show()


In [ ]:
# path = r'D:\Jupyter Notebook\Pinang Interbank Call Money\html embed\single feature check'
# filename = f"{column_to_check}_single_feature_plot.html"
# file_path = os.path.join(path, filename)
# fig.write_html(file_path)
# # fig.write_html(file_path, include_plotlyjs='cdn', full_html=False)

### Overtime Single Bank Checking

In [37]:
# Convert year_quarter to a sortable key (e.g., 2024_q1 -> (2024, 1))
def quarter_sort_key(yq):
    year, q = yq.split('_q')
    return int(year) * 4 + int(q)

In [38]:
# Example: Filter by specific company_name
company_to_plot = "BCA Digital"
df_filtered = combined_df_rasio[combined_df_rasio['company_name'] == company_to_plot].copy()
df_filtered['sort_key'] = df_filtered['year_quarter'].apply(quarter_sort_key)

# Sort the DataFrame by the key
df_filtered = df_filtered.sort_values(by='sort_key')

# Plot with Plotly
fig = px.line(
    df_filtered,
    x='year_quarter',
    y=column_to_check,
    title=f"Value over Time ({company_to_plot})",
    markers=True  # add points
)

fig.update_layout(
    xaxis_title="Year Quarter",
    yaxis_title="Value",
    xaxis=dict(categoryorder='array', categoryarray=df_filtered['year_quarter'])
)

fig.update_yaxes(tickformat=".02%")
fig.show()

### Overtime Multiple Bank Checking

In [39]:
combined_df_rasio['sort_key'] = combined_df_rasio['year_quarter'].apply(quarter_sort_key)
df_sorted = combined_df_rasio.sort_values(by='sort_key')

# --- Step 2: Plot all company_name in one chart ---
fig = px.line(
    df_sorted,
    x='year_quarter',
    y=column_to_check,
    color='company_name',      # Different color for each company_name
    markers=True,            # Show markers on lines
    title="Value Over Time by Company Name"
)

fig.update_layout(
    xaxis_title="Year Quarter",
    yaxis_title=column_to_check,
    xaxis=dict(categoryorder='array', categoryarray=df_sorted['year_quarter'].unique())
)

fig.update_yaxes(tickformat=".02%")
fig.show()

# To check Data

In [ ]:
combined_df_rasio

,posisi,company_name,kewajiban_penyediaan_modal_minimum,aset_produktif_bermasalah_dan_aset_non_produktif_bermasalah_terhadap_total_aset_produktif_dan_aset_non_produktif,aset_produktif_bermasalah_terhadap__total_aset_produktif,cadangan_kerugian_penurunan_nilai_(ckpn)_aset_keuangan_terhadap_aset_produktif,npl_gross,npl_net,return_on_asset,return_on_equity,net_interest_margin,biaya_operasional_terhadap_pendapatan_operasional,cost_to_income_ratio,loan_to_deposit_ratio,leverage_ratio,i.__pihak_terkait_(pelanggaran_bmpk),ii._pihak_tidak_terkait_(pelanggaran_bmpk),i.__pihak_terkait_(pelampauan_bmpk),ii._pihak_tidak_terkait_(pelampauan_bmpk),__harian,__rata_rata,b.gwm_valuta_asing_(harian),posisi_devisa_neto_(pdn)_secara_keseluruhan,a.gwm_utama_rupiah,year,quarter,year_quarter,company_date
0,2022-03-31,BCA Digital,2.2741,0.0068,0.0000,0.0018,0.0000,0.0000,-0.0138,-0.0226,0.0316,1.3312,1.4002,0.3861,0.5757,0.0000,0.0000,0.0000,0.0000,0.0769,0.0000,0.0000,0.0000,NaN,2022,q1,2022_q1,BCA Digital_2022q1
1,2022-06-30,BCA Digital,1.6872,0.0053,0.0000,0.0040,0.0000,0.0000,-0.0100,-0.0182,0.0299,1.2070,1.1035,0.4151,0.4754,0.0000,0.0000,0.0000,0.0000,0.0973,0.0000,0.0000,0.0000,NaN,2022,q2,2022_q2,BCA Digital_2022q2
2,2022-09-30,BCA Digital,1.4651,0.0042,0.0000,0.0027,0.0000,0.0000,-0.0030,-0.0061,0.0306,1.0653,1.0195,0.3617,0.4011,0.0000,0.0000,0.0000,0.0000,0.1269,0.0000,0.0000,0.0000,NaN,2022,q3,2022_q3,BCA Digital_2022q3
3,2022-12-31,BCA Digital,0.9691,0.0041,0.0003,0.0056,0.0009,0.0004,-0.0083,-0.0180,0.0336,1.1643,1.0885,0.4733,0.3570,0.0000,0.0000,0.0000,0.0000,0.1221,0.0000,0.0000,0.0000,NaN,2022,q4,2022_q4,BCA Digital_2022q4
4,2023-03-31,BCA Digital,0.9774,0.0044,0.0008,0.0059,0.0027,0.0013,0.0003,0.0008,0.0450,0.9982,0.9311,0.4389,0.3392,0.0000,0.0000,0.0000,0.0000,0.1053,0.0000,0.0000,0.0000,NaN,2023,q1,2023_q1,BCA Digital_2023q1
5,2023-06-30,BCA Digital,0.8303,0.0049,0.0016,0.0066,0.0045,0.0019,0.0008,0.0024,0.0446,0.9887,0.8867,0.4679,0.3136,0.0000,0.0000,0.0000,0.0000,0.1191,0.0000,0.0000,0.0000,NaN,2023,q2,2023_q2,BCA Digital_2023q2
6,2023-09-30,BCA Digital,0.7537,0.0053,0.0021,0.0094,0.0055,0.0019,0.0013,0.0038,0.0488,0.9828,0.8269,0.5210,0.3097,0.0000,0.0000,0.0000,0.0000,0.1191,0.0000,0.0000,0.0000,NaN,2023,q3,2023_q3,BCA Digital_2023q3
7,2023-12-31,BCA Digital,0.7148,0.0060,0.0040,0.0109,0.0110,0.0021,0.0018,0.0116,0.0536,0.9766,0.8123,0.5172,0.2874,0.0000,0.0000,0.0000,0.0000,0.1350,0.0000,0.0000,0.0000,NaN,2023,q4,2023_q4,BCA Digital_2023q4
8,2024-03-31,BCA Digital,0.6538,0.0072,0.0054,0.0119,0.0181,0.0037,0.0071,0.0227,0.0597,0.9211,0.5075,0.4501,0.2701,0.0000,0.0000,0.0000,0.0000,0.1108,0.0000,0.0000,0.0000,NaN,2024,q1,2024_q1,BCA Digital_2024q1
9,2024-06-30,BCA Digital,0.5820,0.0068,0.0051,0.0110,0.0150,0.0028,0.0074,0.0198,0.0614,0.9228,0.5590,0.5073,0.2639,0.0000,0.0000,0.0000,0.0000,0.1055,0.1039,0.0000,0.0000,NaN,2024,q2,2024_q2,BCA Digital_2024q2


In [21]:
combined_df_rasio.dtypes

posisi                                                                                                              datetime64[ns]
company_name                                                                                                                object
kewajiban_penyediaan_modal_minimum                                                                                         float64
aset_produktif_bermasalah_dan_aset_non_produktif_bermasalah_terhadap_total_aset_produktif_dan_aset_non_produktif           float64
aset_produktif_bermasalah_terhadap__total_aset_produktif                                                                   float64
cadangan_kerugian_penurunan_nilai_(ckpn)_aset_keuangan_terhadap_aset_produktif                                             float64
npl_gross                                                                                                                  float64
npl_net                                                                            